<a href="https://colab.research.google.com/github/JulioSan02/UNCC/blob/4105/HW6/JS_4105_HW6_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import torch.nn as nn
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import torch
import torch.optim as optim

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Data/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [14]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def data_map(x):
    return x.map({'yes': 1, 'no': 0, 'unfurnished': 0, 'semi-furnished': 0.5, 'furnished': 1})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(data_map)
print('All housing data with binary inputs replaced with ones and zeros: \n')
housing.head()

All housing data with binary inputs replaced with ones and zeros: 



,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1.0,0.0,0.0,0.0,1.0,2,1.0,1.0
1,12250000,8960,4,4,4,1.0,0.0,0.0,0.0,1.0,3,0.0,1.0
2,12250000,9960,3,2,2,1.0,0.0,1.0,0.0,0.0,2,1.0,0.5
3,12215000,7500,4,2,2,1.0,0.0,1.0,0.0,1.0,3,1.0,1.0
4,11410000,7420,4,1,2,1.0,1.0,1.0,0.0,1.0,2,0.0,1.0


In [15]:
np.random.seed(0)
train, test = train_test_split(housing, train_size = 0.8, test_size = 0.2, random_state = 100)

scale = StandardScaler()

price_train_og = train.values[:, 0]
price_train_t = torch.tensor(price_train_og)

price_test_og = test.values[:, 0]
price_test_t = torch.tensor(price_test_og)

train_len = len(price_train_og)  # Number of training examples for training set
test_len = len(price_test_og)  # Number of training examples for test set

vars_all  = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

train_stand_og = train
train_stand_og[vars_all] = scale.fit_transform(train[vars_all])

test_stand_og = test
test_stand_og[vars_all] = scale.fit_transform(test[vars_all])

# Inputs using all variables
vars_all_train_stand_og = np.c_[np.ones((train_len, 1)), train_stand_og[vars_all]]
vars_all_train_stand = torch.tensor(vars_all_train_stand_og)

vars_all_test_stand_og = np.c_[np.ones((test_len, 1)), test_stand_og[vars_all]]
vars_all_test_stand = torch.tensor(vars_all_test_stand_og)


In [16]:
def training_loop(n_epochs, optimizer, model, loss_fn, input_train, input_test, output_train, output_test):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(input_train)
    loss_train = loss_fn(t_p_train, output_train)

    t_p_test = model(input_test)

    loss_test = loss_fn(t_p_test, output_test)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch},\tTraining Loss {loss_train.item():.4f},"
            f"\tValidation loss {loss_test.item():.4f}")

In [17]:
seq_model_1a = nn.Sequential(OrderedDict([
            ('hidden_linear', nn.Linear(13, 32)),
            ('hidden_activation', nn.Tanh()),
            ('output_linear', nn.Linear(32, 1))]))
seq_model_1a.double()

optimizer_1a = optim.SGD(seq_model_1a.parameters(), lr = 1e-3)

In [18]:
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1a,
    model = seq_model_1a,
    loss_fn = nn.MSELoss(),
    input_train = vars_all_train_stand,
    input_test = vars_all_test_stand,
    output_train = price_train_t,
    output_test = price_test_t
)

#print('output', seq_model_1a(vars_all_train_stand))
#print('answer', price_test_t)
#print('hidden', seq_model_1a.hidden_linear.weight.grad)

Epoch 1,	Training Loss 26477598436253.3242,	Validation loss 25158763132320.7344


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1162
Epoch 2000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1162
Epoch 3000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1162
Epoch 4000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1162
Epoch 5000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1162


In [19]:
sum([p.numel() for p in seq_model_1a.parameters()])

481

In [20]:
seq_model_1b = nn.Sequential(OrderedDict([
            ('hidden_linear_1', nn.Linear(13, 32)),
            ('hidden_activation_1', nn.Tanh()),
            ('hidden_linear_2', nn.Linear(32, 64)),
            ('hidden_activation_2', nn.Tanh()),
            ('hidden_linear_3', nn.Linear(64, 16)),
            ('hidden_activation_3', nn.Tanh()),
            ('output_linear', nn.Linear(16, 1))
]))

seq_model_1b.double()

optimizer_1b = optim.SGD(seq_model_1b.parameters(), lr = 1e-3)

In [21]:
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1b,
    model = seq_model_1b,
    loss_fn = nn.MSELoss(),
    input_train = vars_all_train_stand,
    input_test = vars_all_test_stand,
    output_train = price_train_t,
    output_test = price_test_t
)

#print('output', seq_model_1b(vars_all_train_stand))
#print('answer', price_test_t)
#print('hidden', seq_model_1b.hidden_linear_3.weight.grad)

Epoch 1,	Training Loss 26477598601991.8906,	Validation loss 25158763249267.4375
Epoch 1000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1069
Epoch 2000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1069
Epoch 3000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1069
Epoch 4000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1069
Epoch 5000,	Training Loss 3478581051101.6851,	Validation loss 3550505829505.1069


In [22]:
sum([p.numel() for p in seq_model_1b.parameters()])

3617